# Data wrangling

Since the data tables on the [Footballguys](https://www.footballguys.com/) website are fully rendered in HTML, we might be able to scrape the data without too much trouble. This gives us good control over exactly what data we download and an easy mechanism by which to update it throughout the season. Let's give it a try using [urllib](https://docs.python.org/3/howto/urllib2.html) and [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/).

In [1]:
# Standard library imports
import time
import urllib.request
from itertools import product
from random import randrange

# PyPI imports
import pandas as pd
from bs4 import BeautifulSoup

# Turn on copy-on-write mode
pd.options.mode.copy_on_write=True

# Decide if we want to re-download the data or not
download_data=False

# Set the data file paths
raw_data_path='../data/raw_qb_data.parquet'
parsed_data_path='../data/parsed_qb_data.parquet'

## 1. Download and parse HTML data

The available data spans 1996 to 2024 and each year has 18 weeks of data. We also will want to download the data for multiple positions. But, let's start with just one. We also need to pick a scoring scheme, let's go with PPR. We can easily change this later. We will use a loop to construct and download the URL for each year and week and parse and collect the data as we get it.

**Note**: Downloading all of the data for one position takes just over 45 minutes.

### 1.1. Download function

In [2]:
def download_url(url: str) -> bytes:
    '''Takes string url, downloads URL and returns HTML bytes object'''

    headers={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Host": "httpbin.io",
        "Sec-Ch-Ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "Sec-Ch-Ua-Mobile": "?0",
        "Sec-Ch-Ua-Platform": '"Linux"',
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "cross-site",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
    }

    # Create the request
    request_params = urllib.request.Request(
        url=url,
        headers=headers
    )   

    # Get the html
    with urllib.request.urlopen(request_params) as response:
        html=response.read()

    return html

### 1.2. HTML parsing function

In [3]:
def parse_html_table(html: bytes, year: int, week: int, position: str, profile: str) -> pd.DataFrame:
    '''Takes a html bytes object from URL, parses data table, adds
    year, week, position and scoring profile and returns as pandas dataframe'''

    # Extract the table rows
    soup=BeautifulSoup(html, 'html.parser')
    table=soup.find('table',{'class':'datasmall table'})
    table_rows=table.find_all('tr')

    # Get the column names from the first row
    columns=table_rows[0].find_all('th')
    column_names=[column.getText() for column in columns]
    column_names.extend(['Position', 'Year', 'Week', 'Scoring profile'])

    # Get the values for each row
    data=[]

    for row in table_rows[1:]:
        columns=row.find_all('td')
        values=[column.getText() for column in columns]
        values.extend([position, year, week, profile])
        data.append(values)

    # Convert to pandas dataframe and return
    return pd.DataFrame(columns=column_names, data=data)

### 1.3. Main download loop

In [4]:
# Main script to download data
download_data = True  # Or False, depending on what you want to do

if download_data is True:
    positions = ['qb', 'rb', 'wr', 'te']
    profile = 'p'
    years = list(range(2020, 2024))
    weeks = list(range(1, 19))

    results = []

    for position, year, week in product(positions, years, weeks):
        print(f'Downloading {position.upper()}, {year}, week {week}')
        url = f'https://www.footballguys.com/playerhistoricalstats?pos={position}&yr={year}&startwk={week}&stopwk={week}&profile={profile}'
        
        # Get the HTML
        html = download_url(url)
        
        # Parse the HTML
        result = parse_html_table(html, position, year, week, profile)
        
        # Collect the result
        results.append(result)

        # Wait before downloading the next page
        time.sleep(randrange(1, 5))

    # Combine the week-by-week dataframes
    data_df = pd.concat(results)

elif download_data is False:
    data_df = pd.read_parquet('../data/raw_qb_data.parquet')
    print('Loaded data from file')

# View the resulting DataFrame
print(data_df.head())

Loaded data from disk


## 2. Fix the player name/team column

In [5]:
test_df = data_df.copy()
test_df[['Player', 'Team']] = test_df['Name'].str.extract(r'^(.*?\b(?:I{1,3}|IV)?)(?:\s+)([A-Z]{2,3})$')
test_df.drop(columns=['Name'], inplace=True)
test_df.rename(columns={'Position': 'Week', 'Year': 'Position', 'Week': 'Year'}, inplace=True)
test_df.head()

,Rank,Name,Age,Exp,G,Cmp,Att,Cm%,PYd,Y/Att,...,Rsh,RshYd,RshTD,FP/G,FantPt,Position,Year,Week,Scoring profile,Team
0,1,Brett Favre,27.0,6.0,1,20,27,74.1,247,9.15,...,1,1,0,26.0,26.0,qb,1996,1,p,GB
1,2,Mark Brunell,26.0,3.0,1,20,31,64.5,212,6.84,...,10,41,0,20.6,20.6,qb,1996,1,p,JAX
2,3,Vinny Testaverde,33.0,10.0,1,19,33,57.6,254,7.70,...,8,42,1,20.4,20.4,qb,1996,1,p,BAL
3,4,Rodney Peete,30.0,8.0,1,20,34,58.8,269,7.91,...,6,10,0,17.8,17.8,qb,1996,1,p,PHI
4,5,Kerry Collins,24.0,2.0,1,17,31,54.8,198,6.39,...,1,14,0,17.3,17.3,qb,1996,1,p,CAR


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17938 entries, 0 to 17937
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Rank             17938 non-null  object
 1   Name             17938 non-null  object
 2   Age              17938 non-null  object
 3   Exp              17938 non-null  object
 4   G                17938 non-null  object
 5   Cmp              17938 non-null  object
 6   Att              17938 non-null  object
 7   Cm%              17938 non-null  object
 8   PYd              17938 non-null  object
 9   Y/Att            17938 non-null  object
 10  PTD              17938 non-null  object
 11  Int              17938 non-null  object
 12  Rsh              17938 non-null  object
 13  RshYd            17938 non-null  object
 14  RshTD            17938 non-null  object
 15  FP/G             17938 non-null  object
 16  FantPt           17938 non-null  object
 17  Position         17938 non-null

## 3. Data reshaping

Now let's get to work transforming this data. We will treat each player as an independent sample within each year, this will give us a set of time series for quarterback performance across a season. The data structure here might be ragged, meaning that the rows are not all the same length. But, I think that is better than trying to put in zeros or placeholder values for the weeks that a player didn't play. The best data structure for this will be a Pandas dataframe with a multi-level index. The index levels will be: player, season (year) and week.

In [18]:
indexes=[
    data_df['Year'].tolist(),
    data_df['Name'].tolist(),
    data_df['Week'].tolist()
]

index_tuples=list(zip(*indexes))
index=pd.MultiIndex.from_tuples(index_tuples, names=['Season', 'Player', 'Week'])
data_df.set_index(index, inplace=True)

# Drop unnecessary columns
data_df.drop(['Name', 'Age', 'Position', 'Year', 'Week', 'Scoring profile', 'Team'], axis=1, inplace=True)
data_df.head()

Rank   Exp  G Cmp Att   Cm%  PYd Y/Att PTD Int  \
Season Player           Week                                                  
1996   Brett Favre      1       1   6.0  1  20  27  74.1  247  9.15   4   0   
       Mark Brunell     1       2   3.0  1  20  31  64.5  212  6.84   2   0   
       Vinny Testaverde 1       3  10.0  1  19  33  57.6  254  7.70   0   0   
       Rodney Peete     1       4   8.0  1  20  34  58.8  269  7.91   2   1   
       Kerry Collins    1       5   2.0  1  17  31  54.8  198  6.39   2   0   

                             Rsh RshYd RshTD  FP/G FantPt  
Season Player           Week                               
1996   Brett Favre      1      1     1     0  26.0   26.0  
       Mark Brunell     1     10    41     0  20.6   20.6  
       Vinny Testaverde 1      8    42     1  20.4   20.4  
       Rodney Peete     1      6    10     0  17.8   17.8  
       Kerry Collins    1      1    14     0  17.3   17.3

In [19]:
# Save the parsed data
data_df.to_parquet(parsed_data_path)